## <center>Applied Data Science Capstone</center>
### <center>Assigment week 3</center>

Getting and preparing the data from the web site Wikipedia

Importing the libraries

In [1]:
import requests
import numpy as np
import lxml.html as lh
import pandas as pd
import numpy as np
import re

print('Libraries imported.')

Libraries imported.


Scraping the web to get the data of the table Postal Codes of the Toronto neibourhoods

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under tree
tree = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = tree.xpath('//tr')
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

Getting the header of the table of postal codes

In [3]:
#Create empty list
col=[]
i=0 #For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


Creating a list with all the data in the tree to later transform to a dataframe

In [4]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

Let's see if all the column have the same amount of elements

In [5]:
[len(C) for (title,C) in col]

[288, 288, 288]

Creating the dataframe

In [6]:
Dict={title:column for (title,column) in col}
postcodes_df=pd.DataFrame(Dict)
postcodes_df.columns = ['Postcode','Borough','Neighbourhood']
postcodes_df.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

Drop all rows with 'Not Assign' string in column Borough

In [7]:
postcodes_df.drop(postcodes_df[postcodes_df['Borough'] == 'Not assigned'].index,inplace = True)
postcodes_df.sort_values(by = ['Postcode'], inplace = True)
postcodes_df.shape

(211, 3)

Delete '\n' character of all values of the dataframe, replace the string 'Not assigned' in column Neighbourhood with the value of 'Borough' column, sort and re index the Data frame

In [9]:
for index, row in postcodes_df.iterrows():
    if postcodes_df.loc[index,'Neighbourhood'] == 'Not assigned':
        postcodes_df.loc[index,'Neighbourhood'] = postcodes_df.loc[index,'Borough']
postcodes_df['Neighbourhood'] = postcodes_df['Neighbourhood'].map(lambda x: re.sub(r'\n', '', x))
postcodes_df.sort_values(by = ['Postcode'], inplace = True)
postcodes_df = postcodes_df.reset_index(drop=True)
postcodes_df.head(30)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge
1,M1B,Scarborough,Malvern
2,M1C,Scarborough,Port Union
3,M1C,Scarborough,Rouge Hill
4,M1C,Scarborough,Highland Creek
5,M1E,Scarborough,Guildwood
6,M1E,Scarborough,Morningside
7,M1E,Scarborough,West Hill
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae


Find the neighborhood with the same postal area and combine the rows

In [10]:
for index, row in postcodes_df.iterrows():
    i = index
    if (index) == postcodes_df.index[-1]:
        break
    else: 
        while (postcodes_df['Postcode'].iloc[index] == postcodes_df['Postcode'].iloc[i + 1]):
            postcodes_df['Neighbourhood'].iloc[index] =  postcodes_df['Neighbourhood'].iloc[index] + ', ' + postcodes_df['Neighbourhood'].iloc[i + 1]
            postcodes_df['Postcode'].iloc[i + 1] = np.nan
            i = i + 1
postcodes_df.dropna(inplace = True)
print(postcodes_df.shape)
postcodes_df.head(50)

(103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
2,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
5,M1E,Scarborough,"Guildwood, Morningside, West Hill"
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae
10,M1J,Scarborough,Scarborough Village
11,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
14,M1L,Scarborough,"Golden Mile, Oakridge, Clairlea"
17,M1M,Scarborough,"Cliffcrest, Scarborough Village West, Cliffside"
20,M1N,Scarborough,"Cliffside West, Birch Cliff"


Number of rows and columns in the final dataframe of Postal Codes for Toronto

In [11]:
print(postcodes_df.shape)

(103, 3)
